# Branch and Bound - Método de Busca Otimizada

## Projeto de Alto Nível

1. **Entrada e Saída**

    **Entrada**: Lista de sequências de DNA e comprimento do motif a ser encontrado.

    **Saída**: Posições dos melhores motifs identificados e a pontuação do alinhamento.

2. **Componentes Principais**

    **Inicialização**: Define variáveis de estado e inicia a busca com offsets zerados.

    **Geração de Ramos**: Explora todas as possibilidades de alinhamento de motifs dentro dos limites da sequência.

    **Avaliação Parcial**: Calcula a pontuação dos motifs formados até um certo nível da busca.

    **Bound**: Descarta ramos cujas pontuações parciais sejam inferiores ao melhor resultado encontrado até o momento.

    **Critério de Paragem**: A busca termina quando todos os offsets forem preenchidos ou quando um ramo for eliminado pela poda.

3. **Fluxo de Dados**: Entrada das sequências e comprimento do motif → Inicialização das variáveis → Exploração iterativa das possibilidades → Cálculo da pontuação parcial e aplicação da poda → Retorno das melhores posições e pontuação final.

## Projeto de Baixo Nível

1. **Cálculo da Pontuação (score_bb)**

    **Descrição**: Determina a qualidade do alinhamento atual, contando os nucleotídeos mais frequentes em cada posição do motif.

    **Algoritmo**:

        Extrai os segmentos de DNA correspondentes aos offsets escolhidos.

        Para cada coluna do alinhamento, conta a ocorrência de cada nucleotídeo.

        A pontuação é a soma das frequências do nucleotídeo mais comum em cada coluna.

2. **Exploração Branch and Bound (branch_and_bound)**

    **Descrição**: Implementa a estratégia de busca Branch and Bound, iterativamente expandindo os offsets e eliminando soluções subótimas.

    **Algoritmo**:

        Se todos os offsets foram definidos, calcula a pontuação final.

        Se a pontuação atual for maior que a melhor encontrada, atualiza o estado global.

        Para cada posição possível na sequência:
        
            Define o offset correspondente.

            Calcula a pontuação parcial para prever a melhor pontuação possível.

        Se a previsão não ultrapassa a melhor pontuação encontrada, descarta o ramo.

        Caso contrário, continua a busca recursivamente.

3. **Execução do Algoritmo Principal (motif_bb)**

    **Descrição**: Função de alto nível que inicializa o estado global e chama o algoritmo de Branch and Bound.

    **Algoritmo**:

        Verifica se todas as sequências possuem o mesmo comprimento.

        Inicializa as variáveis de estado.

        Chama a função branch_and_bound para explorar as soluções.

        Retorna as melhores posições de motifs e sua pontuação.

4. **Saída e Visualização**

    Exibe a melhor configuração de posições encontradas.

    Mostra os segmentos extraídos das sequências que representam os motifs identificados.

    Retorna a pontuação do melhor alinhamento obtido.

In [ ]:
def score_bb(seqs, offset, tam_motif):
    snips = [s[p: p + tam_motif] for p, s in zip(offset, seqs)]
    return sum(max(col.count(x) for x in set(col)) for col in zip(*snips))

def branch_and_bound(offset, num_seqs, limite, tam_motifs, estado_global, seqs, level=0):
    """ Implementação de Branch and Bound para busca do melhor motif """
    
    if level == num_seqs:  # Todos os offsets foram preenchidos
        atual = score_bb(seqs, offset, tam_motifs)
        
        # Se for melhor que o maior score encontrado, atualizar
        if atual > estado_global["maior_s"]:
            estado_global["maior_s"] = atual
            estado_global["melhor_pos"] = offset[:]
        return

    # Gerar Branch
    for pos in range(limite):
        offset[level] = pos
        atual = score_bb(seqs, offset[:level+1] + [0] * (num_seqs - level - 1), tam_motifs)

        # Bound: Se o score máximo possível já for pior, descartar o ramo
        limite_superior = atual + (num_seqs - level - 1) * tam_motifs
        if limite_superior > estado_global["maior_s"]:
            branch_and_bound(offset, num_seqs, limite, tam_motifs, estado_global, seqs, level+1)

def motif_bb(seqs, num_seqs, tam_seq, tam_motif):
    assert all(len(s) == tam_seq for s in seqs)

    estado_global = {"maior_s": 0, "melhor_pos": None}
    offset = [0] * num_seqs
    limite = tam_seq - tam_motif + 1

    branch_and_bound(offset, num_seqs, limite, tam_motif, estado_global, seqs)

    return estado_global["melhor_pos"], estado_global["maior_s"]
